# Word Embeddings : le modèle Word2Vec

## Imports

In [1]:
import sys

from gensim.models.phrases import Phrases, Phraser
from gensim.models import Word2Vec

import nltk
from nltk.tokenize import wordpunct_tokenize
from unidecode import unidecode

## Chargement et traitement des phrases du corpus

### Création d'un objet qui *streame* les lignes d'un fichier pour économiser de la RAM

In [2]:
class MySentences(object):
    """Tokenize and Lemmatize sentences"""
    def __init__(self, filename):
        self.filename = filename

    def __iter__(self):
        for line in open(self.filename, encoding='utf-8', errors="backslashreplace"):
            yield [unidecode(w.lower()) for w in wordpunct_tokenize(line)]

In [4]:
infile = f"../../data/sents.txt"
sentences = MySentences(infile)

### Détection des bigrams

Article intéressant sur le sujet : https://towardsdatascience.com/word2vec-for-phrases-learning-embeddings-for-more-than-one-word-727b6cf723cf

In [5]:
bigram_phrases = Phrases(sentences)

In [6]:
type(bigram_phrases.vocab)

dict

In [10]:
bigram_phraser = Phraser(phrases_model=bigram_phrases)

### Extraction des trigrams

Nous répétons l'opération en envoyant cette fois la liste de bigrams afin d'extraire les trigrams.

In [11]:
trigram_phrases = Phrases(bigram_phraser[sentences])

In [12]:
trigram_phraser = Phraser(phrases_model=trigram_phrases)

### Création d'un corpus d'unigrams, bigrams, trigrams

In [13]:
corpus = list(trigram_phraser[bigram_phraser[sentences]])

## Entrainement d'un premier modèle Word2Vec sur ce corpus (window = 5 ; min_count = 5))

In [14]:
%%time
model = Word2Vec(
    corpus, # On passe le corpus de ngrams que nous venons de créer
    vector_size=32, # Le nombre de dimensions dans lesquelles le contexte des mots devra être réduit, aka. vector_size
    window=5, # La taille du "contexte", ici 5 mots avant et 5 après le mot observé
    min_count=5, # On ignore les mots qui n'apparaissent pas au moins 5 fois dans le corpus (on peut le mettre à plus avec un très gros corpus)
    workers=4, # Permet de paralléliser l'entraînement du modèle en 4 threads
    epochs=5 # Nombre d'itérations du réseau de neurones sur le jeu de données pour ajuster les paramètres avec la descente de gradient, aka. epochs.
)

CPU times: total: 25min 33s
Wall time: 13min 23s


### Sauver le modèle dans un fichier

In [15]:
outfile = f"../../data/tp3modèle1.model"
model.save(outfile)

## Explorer le modèle

### Charger le modèle en mémoire

In [91]:
model = Word2Vec.load("../../data/tp3modèle1.model")

### Calculer la similarité entre deux termes

In [102]:
model.wv.similarity("ministre", "gouvernement")

0.82690287

In [100]:
model.wv.similarity("bruxelles", "capitale")

0.23835611

In [99]:
model.wv.similarity("football", "sport")

0.80381954

### Chercher les mots les plus proches d'un terme donné

In [98]:
model.wv.most_similar("energie", topn=10)

[('efficacite', 0.8009935021400452),
 ('unite', 0.7881107330322266),
 ('action', 0.7860397100448608),
 ('habilete', 0.7850760221481323),
 ('economie', 0.7823023796081543),
 ('armature', 0.7791286110877991),
 ('hypocrisie', 0.7789782881736755),
 ('ingratitude', 0.7773498892784119),
 ('faiblesse', 0.7729592323303223),
 ('indifference', 0.7712991833686829)]

In [97]:
model.wv.most_similar("capitale", topn=10)

[('principaute', 0.9286186695098877),
 ('colonie_belge', 0.8750392198562622),
 ('peninsule', 0.8721823692321777),
 ('legion_etrangere', 0.8436046838760376),
 ('crete', 0.8430392146110535),
 ('section_belge', 0.8391808271408081),
 ('region_parisienne', 0.8371219635009766),
 ('famille_royale', 0.8368630409240723),
 ('rhenanie', 0.8297626376152039),
 ('mer_rouge', 0.828758716583252)]

In [96]:
model.wv.most_similar("colonie", topn=10)

[('guerro', 0.8601323366165161),
 ('demobilisation', 0.850451648235321),
 ('communaute', 0.8402085900306702),
 ('ruhr', 0.8351300954818726),
 ('guorre', 0.8123477101325989),
 ('nation', 0.81203693151474),
 ('collectivite', 0.8118950724601746),
 ('marine_marchande', 0.8092591762542725),
 ('cession', 0.8083475232124329),
 ('mobilisation', 0.8056179881095886)]

### Faire des recherches complexes à travers l'espace vectoriel

In [95]:
print(model.wv.most_similar(positive=['sport', 'cyclisme'], negative=['football']))

[('rallye', 0.8309959173202515), ('ski', 0.8088762760162354), ('marathon', 0.8018616437911987), ('lotus', 0.7980955243110657), ('club', 0.7963196039199829), ('championnat_interclubs', 0.7806680202484131), ('patinage', 0.7763028740882874), ('motocyclisme', 0.7757370471954346), ('challenge', 0.7700577974319458), ('criterium_national', 0.7673411965370178)]


In [93]:
print(model.wv.most_similar(positive=['petrole', 'charbon'], negative=['economie']))

[('dechargement', 0.774098813533783), ('ciment', 0.765225887298584), ('vapeur', 0.753138542175293), ('chargement', 0.7297610640525818), ('wagon', 0.7174645662307739), ('reservoir', 0.7160706520080566), ('oharbon', 0.7151058912277222), ('metal', 0.7108349800109863), ('plancher', 0.710391640663147), ('benzol', 0.7100170254707336)]


In [94]:
print(model.wv.most_similar(positive=['paris', 'londres'], negative=['bruxelles']))

[('rome', 0.8768160939216614), ('berlin', 0.8662904500961304), ('moscou', 0.8549507260322571), ('vienne', 0.8375968933105469), ('pekin', 0.8286409378051758), ('budapest', 0.8105076551437378), ('tokio', 0.8054363131523132), ('teheran', 0.7997197508811951), ('madrid', 0.7921528220176697), ('ankara', 0.7884011268615723)]


 # Exploration Modèle numéro 2 (window = 10 ; min_count = 5)

Entrainer, sauver et charger le modèle 2

In [75]:
%%time
model = Word2Vec(
    corpus, # On passe le corpus de ngrams que nous venons de créer
    vector_size=32, # Le nombre de dimensions dans lesquelles le contexte des mots devra être réduit, aka. vector_size
    window=10, # La taille du "contexte", ici 5 mots avant et 5 après le mot observé
    min_count=5, # On ignore les mots qui n'apparaissent pas au moins 5 fois dans le corpus (on peut le mettre à plus avec un très gros corpus)
    workers=4, # Permet de paralléliser l'entraînement du modèle en 4 threads
    epochs=5 # Nombre d'itérations du réseau de neurones sur le jeu de données pour ajuster les paramètres avec la descente de gradient, aka. epochs.
)

CPU times: total: 28min 32s
Wall time: 14min 38s


In [76]:
outfile = f"../../data/tp3modèle2.model"
model.save(outfile)

model = Word2Vec.load("../../data/tp3modèle2.model")

Calculer la similarité entre deux termes

In [81]:
model.wv.similarity("ministre", "gouvernement")

0.83393437

In [79]:
model.wv.similarity("bruxelles", "capitale")

0.1894651

In [80]:
model.wv.similarity("football", "sport")

0.8331728

Chercher les mots les plus proches d'un terme

In [82]:
model.wv.most_similar("energie", topn=10)

[('action', 0.8092068433761597),
 ('impuissance', 0.8056376576423645),
 ('vigueur', 0.790776789188385),
 ('inertie', 0.7804978489875793),
 ('indifference', 0.7786954641342163),
 ('faiblesse', 0.7742382287979126),
 ('ignorance', 0.7718334794044495),
 ('efficacite', 0.7699581384658813),
 ('inutilite', 0.7671987414360046),
 ('exces', 0.7646012902259827)]

In [83]:
model.wv.most_similar("capitale", topn=10)

[('principaute', 0.9032148122787476),
 ('peninsule', 0.8321616053581238),
 ('vieille_cite', 0.8222121000289917),
 ('colonie_belge', 0.8157285451889038),
 ('crete', 0.8121602535247803),
 ('cite_ardente', 0.8106663823127747),
 ('population', 0.8001872301101685),
 ('banlieue', 0.7910217046737671),
 ('baltique', 0.7852566242218018),
 ('sicile', 0.7839754819869995)]

In [84]:
model.wv.most_similar("colonie", topn=10)

[('cession', 0.7881654500961304),
 ('communaute', 0.7801764011383057),
 ('guerro', 0.7788991332054138),
 ('caisse_autonome', 0.7747275829315186),
 ('guerre', 0.7741776704788208),
 ('collectivite', 0.7708531618118286),
 ('nation', 0.7698078155517578),
 ('marine_marchande', 0.7647090554237366),
 ('dotation', 0.761199414730072),
 ('large_mesure', 0.7594324946403503)]

Faire des recherches complexes à travers l'espace vectoriel

In [85]:
print(model.wv.most_similar(positive=['sport', 'cyclisme'], negative=['football']))

[('rallye', 0.8378145098686218), ('patinage', 0.8234587907791138), ('automobilisme', 0.8085893392562866), ('ski', 0.7935338020324707), ('club', 0.7926409244537354), ('motocyclisme', 0.7887585759162903), ('marathon', 0.7879200577735901), ('championnat_national', 0.7843747735023499), ('tennis', 0.7824083566665649), ('derby', 0.7809351086616516)]


In [87]:
print(model.wv.most_similar(positive=['petrole', 'charbon'], negative=['economie']))

[('ciment', 0.7813999652862549), ('chargement', 0.7581973075866699), ('rails', 0.742189884185791), ('dechargement', 0.7404050230979919), ('vapeur', 0.7351729869842529), ('dragueur', 0.7305387854576111), ('wagon', 0.7304143309593201), ('coke', 0.7244198322296143), ('terril', 0.7223315834999084), ('bris', 0.7195336818695068)]


In [89]:
print(model.wv.most_similar(positive=['paris', 'londres'], negative=['bruxelles']))

[('berlin', 0.8634778261184692), ('rome', 0.8586385846138), ('moscou', 0.8540712594985962), ('vienne', 0.8101162910461426), ('pekin', 0.8076951503753662), ('teheran', 0.7905494570732117), ('tokio', 0.790129542350769), ('ankara', 0.7840481400489807), ('madrid', 0.783541738986969), ('francfort', 0.7780991196632385)]


 # Exploration Modèle numéro 3 (window = 5 ; min_count = 10)

Entrainer, sauver et charger le modèle 3

In [56]:
%%time
model = Word2Vec(
    corpus, # On passe le corpus de ngrams que nous venons de créer
    vector_size=32, # Le nombre de dimensions dans lesquelles le contexte des mots devra être réduit, aka. vector_size
    window=5, # La taille du "contexte", ici 5 mots avant et 5 après le mot observé
    min_count=10, # On ignore les mots qui n'apparaissent pas au moins 5 fois dans le corpus (on peut le mettre à plus avec un très gros corpus)
    workers=4, # Permet de paralléliser l'entraînement du modèle en 4 threads
    epochs=5 # Nombre d'itérations du réseau de neurones sur le jeu de données pour ajuster les paramètres avec la descente de gradient, aka. epochs.
)

CPU times: total: 23min 9s
Wall time: 12min 50s


In [57]:
outfile = f"../../data/tp3modèle3.model"
model.save(outfile)

model = Word2Vec.load("../../data/tp3modèle3.model")

Calculer la similarité entre deux termes

In [58]:
model.wv.similarity("ministre", "gouvernement")

0.82124734

In [59]:
model.wv.similarity("bruxelles", "capitale")

0.26588836

In [60]:
model.wv.similarity("football", "sport")

0.7862419

Chercher les mots les plus proches d'un terme

In [61]:
model.wv.most_similar("energie", topn=10)

[('efficacite', 0.8085280656814575),
 ('indifference', 0.8056556582450867),
 ('habilete', 0.7931762337684631),
 ('indulgence', 0.7835987210273743),
 ('faiblesse', 0.7818674445152283),
 ('ignorance', 0.7805702686309814),
 ('exces', 0.7800633311271667),
 ('hostilite', 0.778734564781189),
 ('avarice', 0.7771517038345337),
 ('action', 0.7739158272743225)]

In [62]:
model.wv.most_similar("capitale", topn=10)

[('principaute', 0.895905077457428),
 ('peninsule', 0.8709556460380554),
 ('colonie_belge', 0.8617537021636963),
 ('region_parisienne', 0.8493494987487793),
 ('population', 0.8359257578849792),
 ('rhenanie', 0.8349437117576599),
 ('crete', 0.8326060771942139),
 ('mediterranee', 0.8311338424682617),
 ('ville', 0.8289176225662231),
 ('vieille_cite', 0.8239152431488037)]

In [63]:
model.wv.most_similar("colonie", topn=10)

[('guerro', 0.8581104874610901),
 ('ruhr', 0.8544015288352966),
 ('communaute', 0.8507928252220154),
 ('marine_marchande', 0.8395102620124817),
 ('collectivite', 0.8279435634613037),
 ('guerre', 0.8265326023101807),
 ('nation', 0.8237548470497131),
 ('guorre', 0.8217140436172485),
 ('demobilisation', 0.8211349248886108),
 ('decentralisation', 0.8162907958030701)]

Faire des recherches complexes à travers l'espace vectoriel

In [64]:
print(model.wv.most_similar(positive=['sport', 'cyclisme'], negative=['football']))

[('rallye', 0.8566862940788269), ('marathon', 0.8264356851577759), ('patinage', 0.811134397983551), ('motocross', 0.8025138974189758), ('club', 0.7957391142845154), ('ski', 0.7812719345092773), ('championnat_national', 0.7808427214622498), ('derby', 0.776512622833252), ('golf', 0.7736311554908752), ('festival', 0.7704788446426392)]


In [66]:
print(model.wv.most_similar(positive=['petrole', 'charbon'], negative=['economie']))

[('ciment', 0.7966238260269165), ('naphte', 0.760569155216217), ('dechargement', 0.7577843070030212), ('gravier', 0.7463268637657166), ('groenland', 0.7358078956604004), ('genievre', 0.730974555015564), ('foin', 0.7282389998435974), ('silex', 0.7197476625442505), ('vapeur', 0.7187607288360596), ('chargement', 0.7146559953689575)]


In [67]:
print(model.wv.most_similar(positive=['paris', 'londres'], negative=['bruxelles']))

[('rome', 0.8686522841453552), ('berlin', 0.8626545667648315), ('moscou', 0.8578198552131653), ('budapest', 0.849730908870697), ('tokio', 0.8311203718185425), ('vienne', 0.815719723701477), ('stockholm', 0.7930417060852051), ('pekin', 0.7902159094810486), ('teheran', 0.7872359752655029), ('madrid', 0.786888599395752)]
